In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
from datetime import datetime
import math

In [2]:
case_data = pd.read_csv("../data/case.csv", index_col=False)
print(case_data.shape)
tenacy_data = pd.read_csv("../data/tenancy.csv", index_col=False)
print(tenacy_data.shape)
vulnerability_data = pd.read_csv("../data/vulnerability.csv", index_col=False)
print(vulnerability_data.shape)

(46301, 7)
(11102, 4)
(4315, 2)


# Merge datasets

In [3]:
#  Convert string dates to dates
tenacy_data['tenancy_start_date_as_date'] = tenacy_data['tenancy_start_date'].apply(lambda x :  datetime.strptime(x, '%d/%m/%Y'))
tenacy_data['tenancy_end_date_as_date'] = tenacy_data['tenancy_end_date'].apply(lambda x :  datetime.now()if str(x) == 'nan' else datetime.strptime(x, '%d/%m/%Y'))
case_data['case_create_date_as_date'] = case_data['case_create_date'].apply(lambda x :  datetime.strptime(x, '%d/%m/%Y %H:%M'))

In [4]:
# Merge tenacy with case data
print(case_data.shape, tenacy_data.shape)
units_cases_df =case_data.merge(tenacy_data, on = 'unit_ref')
units_cases_df = units_cases_df[units_cases_df['case_create_date_as_date'] >= units_cases_df['tenancy_start_date_as_date']]
units_cases_df = units_cases_df[units_cases_df['case_create_date_as_date'] <= units_cases_df['tenancy_end_date_as_date']]
print(units_cases_df.shape)
units_cases_df['time_until_the_case (days)'] = units_cases_df.apply(lambda row :( row['case_create_date_as_date'] - row['tenancy_start_date_as_date']).days, axis = 1 )

# Merge with vulnerability
vulnerability_data_packed = vulnerability_data.groupby(['tenancy_id'])['Vulnerability'].apply(list).reset_index()
units_cases_df = units_cases_df.merge(vulnerability_data_packed, on = 'tenancy_id', how= 'left')
units_cases_df['number of vulnerabilities'] = units_cases_df['Vulnerability'].apply(lambda x : 0 if str(x) == 'nan' else len(x))
units_cases_df['Vulnerability'] = units_cases_df['Vulnerability'].apply(lambda x : x if isinstance(x, list) else ['Healthy'])
print(units_cases_df.shape)

# Vulnerability label
units_cases_df['vulnerability_boolean'] = units_cases_df['number of vulnerabilities'].apply(lambda x : 0 if x == 0 else 1 )

# Create label
units_cases_df['label'] = units_cases_df['case_sub_type'].apply(lambda x : 1 if x == 'Damp & Mould' else 0)

(46301, 8) (11102, 6)
(45364, 13)
(45364, 16)


In [5]:
units_cases_df

,case_number,unit_ref,unit_build_year,case_status,case_type,case_sub_type,case_create_date,case_create_date_as_date,tenancy_id,tenancy_start_date,tenancy_end_date,tenancy_start_date_as_date,tenancy_end_date_as_date,time_until_the_case (days),Vulnerability,number of vulnerabilities,vulnerability_boolean,label
0,Case0,Unit8230,"2,005.0000",Resolved,ASB,NaN,21/02/2019 10:20,2019-02-21 10:20:00,Tenant8773,21/08/2017,NaN,2017-08-21,2022-07-29 13:37:11.185120,549,"[Illness, Mental Health]",2,1,0
1,Case97,Unit8230,"2,005.0000",Resolved,ASB,NaN,22/05/2019 07:08,2019-05-22 07:08:00,Tenant8773,21/08/2017,NaN,2017-08-21,2022-07-29 13:37:11.185120,639,"[Illness, Mental Health]",2,1,0
2,Case99,Unit8230,"2,005.0000",Resolved,Query,NaN,22/05/2019 07:33,2019-05-22 07:33:00,Tenant8773,21/08/2017,NaN,2017-08-21,2022-07-29 13:37:11.185120,639,"[Illness, Mental Health]",2,1,0
3,Case101,Unit8230,"2,005.0000",Resolved,ASB,NaN,22/05/2019 07:59,2019-05-22 07:59:00,Tenant8773,21/08/2017,NaN,2017-08-21,2022-07-29 13:37:11.185120,639,"[Illness, Mental Health]",2,1,0
4,Case1027,Unit8230,"2,005.0000",Resolved,Direct Maintenance,OOH / ECO,13/06/2019 13:33,2019-06-13 13:33:00,Tenant8773,21/08/2017,NaN,2017-08-21,2022-07-29 13:37:11.185120,661,"[Illness, Mental Health]",2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45359,Case46108,Unit8101,"2,018.0000",Resolved,Service Charges,Calculation/Apportionment Query,08/05/2019 09:27,2019-05-08 09:27:00,Tenant7930,30/08/2018,NaN,2018-08-30,2022-07-29 13:37:11.184054,251,[Healthy],0,0,0
45360,Case46143,Unit6914,"2,004.0000",Resolved,Service Charges,Calculation/Apportionment Query,10/05/2019 14:21,2019-05-10 14:21:00,Tenant3261,06/08/2004,NaN,2004-08-06,2022-07-29 13:37:11.178118,5390,[Healthy],0,0,0
45361,Case46182,Unit1190,NaN,Resolved,Service Charges,Invoice/Breakdown Request,14/05/2019 12:59,2019-05-14 12:59:00,Tenant2262,14/05/2004,NaN,2004-05-14,2022-07-29 13:37:11.176980,5478,[Healthy],0,0,0
45362,Case46201,Unit4371,"1,899.0000",Resolved,Income,Direct Debit Enquiry,15/05/2019 09:22,2019-05-15 09:22:00,Tenant6001,13/05/2019,NaN,2019-05-13,2022-07-29 13:37:11.181748,2,"[Illness, Elderly]",2,1,0


# Hypothesis Testing

HO: mean_health == mean_vulnerable (tenants with vulnerability are equally likely to develop damp problems with the healthy tenants)
H1: mean_health != mean_vulnerable

Keep only the columns that we need

In [6]:
limit_df = units_cases_df[['case_number', 'unit_ref', 'tenancy_id', 'vulnerability_boolean', 'label']]
limit_df.shape

(45364, 5)

In [7]:
limit_df.head()

,case_number,unit_ref,tenancy_id,vulnerability_boolean,label
0,Case0,Unit8230,Tenant8773,1,0
1,Case97,Unit8230,Tenant8773,1,0
2,Case99,Unit8230,Tenant8773,1,0
3,Case101,Unit8230,Tenant8773,1,0
4,Case1027,Unit8230,Tenant8773,1,0


In [8]:
p_hat1 = limit_df[(limit_df['label'] == 1) & (limit_df['vulnerability_boolean'] == 1)].shape[0] / limit_df[limit_df['vulnerability_boolean'] == 1].shape[0]

n1 = limit_df[limit_df['vulnerability_boolean'] == 1].shape[0]

In [9]:
p_hat2 = limit_df[(limit_df['label'] == 1) & (limit_df['vulnerability_boolean'] == 0)].shape[0] / limit_df[limit_df['vulnerability_boolean'] == 0].shape[0]

n2 = limit_df[limit_df['vulnerability_boolean'] == 0].shape[0]

In [33]:
p_hat = (n1*p_hat1 + n2*p_hat2)/(n1+n2)

In [34]:
print(p_hat1, n1, p_hat2, n2, p_hat)

0.0391552335440111 12974 0.027878974992281567 32390 0.031103959086500307


In [39]:
def estimate_test_statistic(p_hat1, p_hat2, p_hat, n1, n2):
    nominator = p_hat1 - p_hat2
    denominator = math.sqrt(p_hat*(1-p_hat)*(1/n1 + 1/n2))
    return nominator/denominator

In [42]:
print(f'Test statistic : {estimate_test_statistic(p_hat1, p_hat2, p_hat, n1, n2)}')
if estimate_test_statistic(p_hat1, p_hat2, p_hat, n1, n2) > 1.96: # 95% confidence level
    print("Null hypothesis is rejected and tenants with vulnerability are not as equal as healthy tenants to develop damp problems")
else:
    print("The null hypothesis is not rejected")

Test statistic : 6.251803920079067
Null hypothesis is rejected and tenants with vulnerability are not as equal as healthy tenants to develod damp problems
